In [13]:
import csv
import pandas as pd
import numpy as np
import math

import statistics

In [14]:
dict = {"reg_1" : [], "dur_reg_1" : [], "reg_2" : [], "dur_reg_2" : [], "reg_3" : [], "dur_reg_3" : [], "reg_4" : [], "dur_reg_4" : []}
#In my model I created 4 regions of interest (ROIs) for my stimuli. I create a database which will cound teh number of fixations and their average dration on each ROI (e.g., reg_3 is the #of fixations on ROI3, dur_reg_3 is teh mean duration of those fixations)

In [15]:
df = pd.DataFrame(dict)
  
display(df)

,reg_1,dur_reg_1,reg_2,dur_reg_2,reg_3,dur_reg_3,reg_4,dur_reg_4


In [4]:
def find_fix(x, y, threshold, sep, extra):   #x, y are the mean coordinates of left and right eyes on stimuli, threshhold is the selected number of pixels around the fixation, which is considered part of fixation, sep is the minimum value of diration used, and extra is teh threshhold which says if the next point is part of same fixation or saccade.   
        
    rg, dist =[], []
    
    
    start, end, i = 0, 0, 0
    cur_x, cur_y = [], []
    while i < len(x)- sep: # use: i->i+sep
        x_max, x_min, y_max, y_min = max(x[i:i+sep]), min(x[i:i+sep]), max(y[i:i+sep]), min(y[i:i+sep]) 
        if math.sqrt( (x_max-x_min)**2 + (y_max-y_min)**2 ) <= threshold:
            start = i
            end = i+sep
            while (i < len(x)- sep) and (math.sqrt( (x_max-x_min)**2 + (y_max-y_min)**2 ) <= threshold):
                x_new, y_new = x[end], y[end]
                x_max, x_min, y_max, y_min = max(x_max, x_new), min(x_min, x_new), max(y_max, y_new), min(y_min, y_new) 
                i += 1 
                end += 1                    

            #add
            cur_x.append(statistics.median(x[start:end]))
            cur_y.append(statistics.median(y[start:end]))
                
            rg.append(end - start)
            dist.append(0)
                

        else:
            
            if dist:
                dist[-1] += 1
                
            start, end = i+1, i+1
            i += 1
                    
    x_fin, y_fin, rg_final, dist_final = [], [], [], []
    i = 0
    
    while i < len(cur_x):
        if dist[i] == 0:
            temp_x, temp_y = [] ,[]
            rg_final.append(0)
            while i < len(cur_x) and dist[i] == 0:
                rg_final[-1] += rg[i]
                temp_x.append(cur_x[i])
                temp_y.append(cur_y[i])
                i += 1
            
            if i == len(cur_x):
                x_fin.append(statistics.median(temp_x))
                y_fin.append(statistics.median(temp_y))
                dist_final.append(0)
                break
            
            temp_x.append(cur_x[i])
            temp_y.append(cur_y[i])
            x_fin.append(statistics.median(temp_x))
            y_fin.append(statistics.median(temp_y))
            rg_final[-1] += rg[i]
            dist_final.append(dist[i])
            i+=1
            
            
        else:
            x_fin.append(cur_x[i])
            y_fin.append(cur_x[i])
            rg_final.append(rg[i])
            dist_final.append(dist[i])
            i+=1
    
                
    return x_fin, y_fin, rg_final, dist_final
        
    

In [5]:
def fix_zeros(x, y, threshold1): # I taking care of blinks, threshhold1 is the duration of blinking 
    
    X_new = []
    Y_new = []
    start = end = -1
    zero_mode = False
    for i in range(len(x)):
        
        if not zero_mode:
            if x[i] == 0:
                start = i-1
                zero_mode = True
                
            else:
                X_new.append(x[i])
                Y_new.append(y[i])
                
        else:
            if x[i] == 0:
                if i == len(x)-1:
                    if start == -1: return ([], [])
                    else:
                        if i - start <= threshold1:
                            av_x, av_y = x[start], y[start]
                            for _ in range(start+1, i+1):
                                X_new.append(av_x)
                                Y_new.append(av_y)
            else:
                end = i
                if end - start - 1 <= threshold1:
                    if start == -1:
                        av_x, av_y = x[end], y[end]
                    else:
                        av_x, av_y = 0.5*(x[start]+x[end]), 0.5*(y[start]+y[end])

                    for _ in range(start+1, end):
                        X_new.append(av_x)
                        Y_new.append(av_y)

                zero_mode = False
                X_new.append(x[i])
                Y_new.append(y[i])
                            
    return (X_new, Y_new)
                           

In [6]:
def fixation(path):
    df = pd.read_csv(path)
    
    X = list(df['Average_X'])
    Y = list(df['Average_Y'])
    X, Y = fix_zeros(X, Y, 200) # threshhold 1 = 200
    X, Y, dist, sep = find_fix(X, Y, 47, 100, 20) # 47 pixels part of one fixation, 100 is the minimum duration of a fixation, 20 is the minimum duration of a saccade (helps to eliminate noise or classify saccades). 
    return X, Y, dist, sep

In [18]:
base = 'C:/Users/ufdat/Dropbox (UFL)/Infographic_study/Measurements/Main Learning Task/CSV/' # select your destination
#base = ''
    
a, b, c, d = fixation(base + 'P_12_Qorder_7_T_1.csv') # and file 
print (a) # fixation X coordinate 
print(len(a)) # number of all fixations
print (b) # fixation Y coordinate
print(len(b)) #  number of all fixations 
print (c) # duration of each fixation
print(len(c)) 
print (d) # distance of saccades
print(len(d)) 

[854.25, 529.1625, 454.87500000000006, 842.425, 448.425, 363.25, 453.7, 342.3, 459.125, 522.4125, 606.2874999999999, 424.375, 464.125, 580.7249999999999, 1219.6625, 1292.15, 1189.2875, 1252.1, 1320.15, 1267.3249999999998, 1343.3375, 1215.475, 1382.2, 1149.1875, 1240.6, 1279.1999999999998, 1190.5500000000002, 775.7]
28
[854.25, 410.75, 118.5375, 125.2875, 646.7750000000001, 744.4125000000001, 276.66249999999997, 287.35, 644.6750000000001, 629.975, 621.95, 675.25, 671.95, 662.675, 581.3, 604.275, 601.025, 606.1625, 1320.15, 606.1375, 609.9875, 428.05, 430.875, 664.7625, 634.7375, 607.775, 602.1, 390.17499999999995]
28
[245, 315, 329, 243, 376, 367, 504, 294, 583, 667, 377, 324, 250, 306, 357, 254, 393, 336, 272, 265, 361, 360, 359, 507, 381, 229, 347, 182]
28
[125, 129, 131, 136, 102, 138, 103, 125, 97, 99, 110, 92, 102, 139, 98, 101, 96, 97, 95, 98, 116, 108, 122, 103, 93, 100, 143, 0]
28


In [19]:
import pandas as pd
import numpy as np

a = np.array(a)
b = np.array(b)
c = np.array(c)
d = np.array(d)
e = len(a)

#df = pd.DataFrame({"X_pos" : a, "Y_pos" : b, "Durration" : c, "Saccade length" : d})
#df.to_csv("output.csv", index=False)

In [20]:
reg_1 = []  #classifying to 4 ROIs
reg_2 = [] 
reg_3 = [] 
reg_4 = []
dur_reg_1 = [] 
dur_reg_2 = []
dur_reg_3 = []
dur_reg_4 = [] 

for i in range(e):
    if a[i] <= 840 and b[i] <= 525:#I use the coordinates of x and y as a threshhold for ROIs. It is teh central point. 
            reg_1.append(1)
            dur_reg_1.append(c[i])
            
    elif a[i] > 840 and b[i] <= 525:
            reg_2.append(1)
            dur_reg_2.append(c[i])
    elif a[i] <= 840 and b[i] > 525:
            reg_3.append(1)
            dur_reg_3.append(c[i])
    else:
            reg_4.append(1)
            dur_reg_4.append(c[i])

In [21]:
f = len(reg_1)
g = len(reg_2)
h = len(reg_3)
i = len(reg_4)
j = np.mean(dur_reg_1)
k = np.mean(dur_reg_2)
l = np.mean(dur_reg_3)
m = np.mean(dur_reg_4)

In [22]:
f, j, g, k, h, l, i, m

(5, 324.8, 3, 320.6666666666667, 8, 406.25, 12, 328.9166666666667)

In [12]:
df.loc[len(df.index)] = [f, j, g, k, h, l, i, m]  # I write the records in table 
  
display(df)

,reg_1,dur_reg_1,reg_2,dur_reg_2,reg_3,dur_reg_3,reg_4,dur_reg_4
0,24.0,342.625,22.0,474.590909,26.0,417.615385,23.0,456.217391


In [23]:
df.to_csv("output_new.csv", index=False) # save the file